In [2]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
from sklearn.preprocessing import StandardScaler

In [4]:
random_seed = 2022
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)

In [5]:
data_dir = '/USER/data002'

In [6]:
df = pd.read_csv(os.path.join(data_dir,'train.csv'))

In [7]:
df.head()

,날짜,시간,10,100,101,120,121,140,150,160,...,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,20200101,0,83247,19128,2611,5161,1588,892,32263,1636,...,1311,3482,11299,7072,1176,3810,748,3920,2133,3799
1,20200101,1,89309,19027,3337,5502,1650,1043,35609,1644,...,1162,3849,13180,8771,1283,3763,782,3483,2057,4010
2,20200101,2,66611,14710,2970,4631,1044,921,26821,1104,...,768,2299,7986,5426,1536,3229,491,2634,1526,3388
3,20200101,3,53290,13753,2270,4242,1021,790,21322,909,...,632,1716,5703,3156,1104,2882,431,2488,1268,3686
4,20200101,4,52095,17615,2406,3689,1840,922,22711,1354,...,875,2421,5816,2933,1206,2433,499,2952,1927,5608


In [9]:
scaler = StandardScaler()
scaler.fit(df[df.columns.values[2:]])

StandardScaler()

In [141]:
len(df[df['날짜']!=20200229][df['날짜']!=20200330])/24

136.0

## Data Preprocessing
##### 20200229 & 20200330 제거 후 작업

In [187]:
class CustomDataset(Dataset) :
    def __init__(self, data_dir, mode, seq_len, scaler) :
        self.data_dir = data_dir
        self.mode = mode
        self.seq_len = seq_len*24
        
        df_path = os.path.join(self.data_dir, mode+'.csv')
        df = pd.read_csv(df_path)
        df_col = df.columns.values
        df_scale = scaler.transform(df[df_col[2:]])
        df[df_col[2:]] = df_scale
        
        if mode == 'train' :
            df_train = df[df['날짜']!=20200229][df['날짜']!=20200330]

            self.input_data = []
            self.output_data = []

            for t in range(0,len(df_train)-self.seq_len,24) :
                temp_input_data = []
                temp_output_data = []
                for col in df_col[2:] :
                    road = df_train[col].tolist()
                    temp_input = [float(i) for i in road[t: t+self.seq_len]]
                    temp_output = [float(j) for j in road[t+self.seq_len:t+2*self.seq_len]]
                    temp_input_data.append(temp_input)
                    temp_output_data.append(temp_output)
                self.input_data.append(temp_input_data)
                self.output_data.append(temp_output_data)
                
        elif mode == 'valid' :

            self.input_data = []
            self.output_data = []

            temp_input_data = []
            temp_output_data = []
            for col in df_col[2:] :
                road = df[col].tolist()
                temp_input = [float(i) for i in road[:self.seq_len]]
                temp_output = road[self.seq_len:self.seq_len*2]
                temp_input_data.append(temp_input)
                temp_output_data.append(temp_output)
            self.input_data.append(temp_input_data)
            self.output_data.append(temp_output_data)
            
        elif mode == 'test' :
            
            self.input_data = []

            
            temp_input_data = []
            for col in df_col[2:] :
                road = df[col].tolist()
                temp_input = [float(i) for i in road[:self.seq_len]]
                temp_input_data.append(temp_input)
            self.input_data.append(temp_input_data)
            
    def __getitem__(self, index) :
        day = index // 35
        col = index % 35
        
        if self.mode == 'test' :
            x = torch.tensor(self.input_data[day][col])
            
            return x
        
        x = torch.tensor(self.input_data[day][col])
        label = torch.tensor(self.output_data[day][col])
        if label.shape[0] != 168:
            print('day', 'co', day, col)
        return x, label
    
    def __len__(self) :
        return len(self.input_data)*35

In [11]:
class NBeats(nn.Module) :
    def __init__(self, input_size, hidden_dim, output_size, num_layers=4, batch_size=64, bidirectional=False, dropout=0.3, batch_first=True) :
        super(NBeats, self).__init__()
        self.n_direction = 2 if bidirectional else 1
        self.input_size = input_size
        self.hidden_dim = hidden_dim
        self.output_size = output_size
        self.num_layers = num_layers
        self.batch_first = batch_first

        self.rnn1 = nn.LSTM(input_size = self.input_size,
                            hidden_size = self.hidden_dim,
                            num_layers = self.num_layers,
                            dropout = dropout,
                            bias = True,
                            batch_first = self.batch_first,
                            bidirectional = bidirectional)
        self.rnn2 = nn.LSTM(input_size = self.hidden_dim,
                            hidden_size = self.hidden_dim,
                            num_layers = self.num_layers,
                            dropout = dropout,
                            bias = True,
                            batch_first = self.batch_first,
                            bidirectional = bidirectional)

        self.activation = nn.LeakyReLU(0.2)
            
        self.out = nn.Linear(self.hidden_dim*self.n_direction, self.output_size)
        
        self.h_in = nn.Parameter(torch.zeros(num_layers, 1, hidden_dim, requires_grad=True)).to(device)
        self.c_in = nn.Parameter(torch.zeros(num_layers, 1, hidden_dim, requires_grad=True)).to(device)
        
    def forward(self, x) :
        h_in = self.h_in.expand(self.h_in.shape[0], x.shape[0], self.h_in.shape[2])
        c_in = self.c_in.expand(self.c_in.shape[0], x.shape[0], self.c_in.shape[2])
        
        output, (h1, c1) = self.rnn1(x, (h_in.contiguous(), c_in.contiguous()))
        output = self.activation(output)

        output, (h2, c2) = self.rnn2(output, (h1, c1))
        output = self.activation(output)

        output = self.out(output)

        return output, (h2, c2)

In [12]:
class NBeats_RNN(nn.Module) :
    def __init__(self, input_size, hidden_dim, output_size, mode='gru', num_layers=4, batch_size=64, bidirectional=False, dropout=0.3, batch_first=True) :
        super(NBeats, self).__init__()
        self.mode = mode
        self.n_direction = 2 if bidirectional else 1
        self.input_size = input_size
        self.hidden_dim = hidden_dim
        self.output_size = output_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        if self.mode == 'gru' :
            self.rnn1 = nn.GRU(input_size = self.input_size,
                               hidden_size = self.hidden_dim,
                               num_layers = self.num_layers,
                               dropout = dropout,
                               bias = True,
                               batch_first = self.batch_first,
                               bidirectional = bidirectional)
            self.rnn2 = nn.GRU(input_size = self.hidden_dim,
                               hidden_size = self.hidden_dim,
                               num_layers = self.num_layers,
                               dropout = dropout,
                               bias = True,
                               batch_first = self.batch_first,
                               bidirectional = bidirectional)
        else :
            self.rnn1 = nn.RNN(input_size = self.input_size,
                               hidden_size = self.hidden_dim,
                               num_layers = self.num_layers,
                               dropout = dropout,
                               bias = True,
                               batch_first = self.batch_first,
                               bidirectional = bidirectional)
            self.rnn2 = nn.RNN(input_size = self.hidden_dim,
                               hidden_size = self.hidden_dim,
                               num_layers = self.num_layers,
                               dropout = dropout,
                               bias = True,
                               batch_first = self.batch_first,
                               bidirectional = bidirectional)
        
        self.activation = nn.LeakyReLU(0.2)
            
        self.out = nn.Linear(self.hidden_dim*self.n_direction, self.output_size)
        
        self.h_in = nn.Parameter(torch.zeros(num_layers, 1, hidden_dim, requires_grad=True)).to(device)
        
    def forward(self, x, h0) :
        h_in = self.h_in.expand(self.h_in.shape[0], x.shape[0], self.h_in.shape[2])
        
        output, h1 = self.rnn1(x, h0)
        output = self.activation(output)

        output, h2 = self.rnn2(output, h1)
        output = self.activation(output)

        output = self.out(output)

        return output, h2

In [38]:
# 결과 파일과 모델 가중치 파일 저장을 위해 log 디렉토리 생성. 중요한 파일이 덮어씌워지지 않도록 주의
os.makedirs('log', exist_ok=True)      # log 폴더 생성, 이미 생성되었을 시 추가로 생성하지 않도록 exist_ok=True


def save_model(model_name, model, optimizer):      # 모델 가중치 파일 저장 함수
    state = {
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'scheduler' : scheduler.state_dict()
    }
    torch.save(state, os.path.join('log', model_name + '.pt'))
    print('model saved\n')


def load_model(model_name, model, optimizer=None):      # 모델 가중치 파일 로드 함수
    state = torch.load(os.path.join(model_name))
    model.load_state_dict(state['model'])
    if optimizer is not None:
        optimizer.load_state_dict(state['optimizer'])
    print('model loaded')

In [35]:
dtype = torch.float
num_epochs = 50
learning_rate = 1e-3
seq_len = 7

input_size = 1
hidden_dim = 1024
output_size = seq_len*24
batch_size = 64
num_layers = 4

In [188]:
train_dataset = CustomDataset(data_dir=data_dir, mode='train', scaler=scaler, seq_len=seq_len)
valid_dataset = CustomDataset(data_dir=data_dir, mode='valid', scaler=scaler, seq_len=seq_len)

In [189]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=1, shuffle=False)

In [151]:
len(df[df['날짜']!=20200229][df['날짜']!=20200330])/24 - 4515/35

7.0

In [158]:
length = 0
for i in train_dataset :
    length += 1
    
print(length,len(train_dataset),129*35)

4515 4515 4515


In [199]:
df.iloc[[882,3212]]

,날짜,시간,10,100,101,120,121,140,150,160,...,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
882,20200206,18,0,86,0,1,0,0,4,0,...,49,0,0,4,0,0,0,35,0,0
3212,20200515,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [157]:
print(f'Train Dataset : {len(train_dataset)}, Valid Dataset : {len(valid_dataset)}')

Train Dataset : 4515, Valid Dataset : 35


In [180]:
for sample in train_dataloader :
    input_data, output_data = sample
    print(input_data.shape,output_data.shape)
    break

128 23
123 12
125 1


RuntimeError: stack expects each tensor to be equal size, but got [168] at entry 0 and [24] at entry 26

In [36]:
model = NBeats(input_size=input_size, hidden_dim=hidden_dim, output_size=output_size, batch_size=batch_size, num_layers=num_layers).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler =  torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=1e-4, epochs=num_epochs, steps_per_epoch=len(train_dataloader))

In [21]:
print(model)

NBeats(
  (rnn1): LSTM(1, 1024, num_layers=4, batch_first=True, dropout=0.3)
  (rnn2): LSTM(1024, 1024, num_layers=4, batch_first=True, dropout=0.3)
  (activation): LeakyReLU(negative_slope=0.2)
  (out): Linear(in_features=1024, out_features=168, bias=True)
)


In [37]:
train_batch_loss = []
train_epoch_loss = []

valid_epoch_loss = []
valid_min_epoch_loss = np.inf
for epoch in tqdm(range(num_epochs)) :
    model.train()

    for i, sample in enumerate(train_dataloader) :

        input_data, output_data = sample

        input_data = input_data.unsqueeze(2).to(device)
        output_data = output_data.unsqueeze(2).to(device)

        pred, (h_out, c_out) = model(input_data)
        loss = criterion(pred, output_data)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        train_batch_loss.append(loss.item())
        train_epoch_loss.append(loss.item())

        if i % 400 == 399:      # 400개의 batch마다 training Loss 출력
            print('Train Epoch: {:2} | Batch: {:4} | Loss: {:1.2f}'.format(epoch+1, i+1, np.array(train_batch_loss).mean()))
            train_batch_loss = []

    print(f'Train Epoch_Loss: {np.array(train_epoch_loss).mean()}')
    train_epoch_loss = []

    model.eval()      # 모델을 eval mode로 전환. eval mode에서 적용되면 안되는 drop out 등이 적용되지 않게 하기 위함

    with torch.no_grad():      # validation / test set에 대해서는 weight 및 bias의 update, 즉, gradient descent가 일어나지 않도록 no_grad()를 선언

        for i, sample in enumerate(valid_dataloader):      # enumerate 함수를 통해 validate_dataloader에서 'batch의 index'와 'batch'를 순서대로 호출

            input_data, output_data = sample      # validate_dataloader에서 불러온 sample은 [[날짜, 시간], [도로], [[input_data],[output_data]]]로 구성됨. validation에는 [[input_data], [output_data]]만 사용

            input_data = input_data.unsqueeze(2).to(device)
            output_data = output_data.unsqueeze(2).to(device)

            pred, (h_in, c_in) = model(input_data)
            loss = criterion(pred, output_data)
            valid_epoch_loss.append(loss.item())

        print('\nValid Epoch: {:2} | Loss: {:1.2f}'.format(epoch+1, np.array(valid_epoch_loss).mean()))

        if np.array(valid_epoch_loss).mean() < valid_min_epoch_loss:
            save_model('nbeats_v001', model, optimizer)
            valid_min_epoch_loss = np.array(valid_epoch_loss).mean()

        valid_epoch_loss = []

  0% 0/50 [00:00<?, ?it/s]

Train Epoch_Loss: 0.9651177128156027

Valid Epoch:  1 | Loss: 0.95


  2% 1/50 [00:07<06:25,  7.87s/it]

model saved

Train Epoch_Loss: 0.9445599516232809

Valid Epoch:  2 | Loss: 0.95


  4% 2/50 [00:15<06:18,  7.89s/it]

...

Train Epoch_Loss: 0.17281142373879751

Valid Epoch: 48 | Loss: 0.13


 98% 49/50 [04:19<00:03,  3.25s/it]

Train Epoch_Loss: 0.18293361365795135

Valid Epoch: 49 | Loss: 0.13


100% 50/50 [04:23<00:00,  5.26s/it]

Train Epoch_Loss: 0.14350182811419168

Valid Epoch: 50 | Loss: 0.13


In [66]:
print(output_data.squeeze(2))

tensor([[-1.2267, -1.4794, -1.5796, -1.5635, -1.2612, -0.3431,  0.6267,  1.0206,
          0.7424,  0.5600,  0.6943,  0.7258,  0.5829,  0.5568,  0.6683,  0.7546,
          0.5960,  0.7031,  0.6144,  0.0128, -0.3113, -0.4340, -0.7481, -1.1034,
         -1.3319, -1.4832, -1.5597, -1.5476, -1.3988, -0.8801, -0.0803,  0.4733,
          0.5977,  0.5413,  0.6624,  0.6525,  0.5293,  0.6534,  0.6787,  0.8033,
          0.8950,  1.0185,  0.7473,  0.1111, -0.1745, -0.3250, -0.5926, -0.9449,
         -1.2490, -1.4348, -1.5124, -1.5163, -1.3237, -0.6496,  0.2151,  0.7481,
          0.7963,  0.7745,  0.8858,  0.8497,  0.8047,  0.7751,  0.8081,  0.8975,
          1.0264,  1.2133,  0.8903,  0.1779, -0.1200, -0.2408, -0.5797, -0.8939,
         -1.2127, -1.4307, -1.5251, -1.5112, -1.3321, -0.6978,  0.1535,  0.6991,
          0.8203,  0.7404,  0.9182,  0.8872,  0.7465,  0.8051,  0.8417,  0.9513,
          1.0337,  1.1850,  0.9802,  0.3604,  0.0785, -0.0921, -0.4629, -0.8650,
         -1.2112, -1.4070, -

In [39]:
submission_file_path = os.path.join(data_dir,'sample_submission.csv')
submission_table = pd.read_csv(submission_file_path)

In [87]:
test_dataset = CustomDataset(data_dir=data_dir, mode='test', scaler=scaler, seq_len=seq_len)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [88]:
length = 0
for sample in test_dataloader :
    length += 1
    
print(length)

1


In [62]:
with torch.no_grad():
    for sample in test_dataloader :
        input_data = sample.unsqueeze(2).to(device)
        pred, (h_out, c_out) = model(input_data)
        print(pred.squeeze(0))

torch.Size([168, 168])
